In [1]:
import rasterio as rio
from rasterio import warp
import numpy as np
import matplotlib.pyplot as plt

from rasterio.windows import Window
from rasterio.transform import Affine

from shapely.geometry import box

In [2]:
import os
import matplotlib.patches as mpatches
import ast
from tqdm import tqdm
import gc

In [3]:
rgb_path = "satellite/rgb/"
scl_path = "satellite/classifications/"
rgb_files = os.listdir(rgb_path)
scl_files = os.listdir(scl_path)

In [4]:
# align classfication data to rgb data because of different spatial resolution
def align(src, dst, src_arr, shape):
    #src_new = np.repeat(np.zeros_like(dst_arr[[0]]), src.count, 0)
    src_new = np.repeat(np.zeros(shape, dtype=np.uint8), src.count, 0)
    geom = warp.reproject(
        src_arr, src_new, 
        src_crs=src.crs, dst_crs=dst.crs,
        src_transform=src.transform, dst_transform=dst.transform,
        dst_resolution=dst.res,
        resampling=warp.Resampling.nearest
    )
    return src_new, geom

In [5]:
lc = rio.open("denmark_landcover.tif")

In [6]:
lc_r = lc.read()
shape = lc_r[[0]].shape

In [7]:
shape

(1, 41742, 40200)

In [14]:
rgb = rio.open(rgb_path + rgb_files[1])

In [15]:
s = rgb.read()[[0]].shape

In [16]:
lc_a,geom2 = align(lc, rgb, lc_r, s)

In [17]:
geom2

(array([[[ 20,  20,  20, ..., 210, 210, 210],
         [ 20,  20,  20, ..., 210, 210, 210],
         [ 20,  20,  20, ..., 253, 253, 210],
         ...,
         [ 20,  20,  20, ..., 253, 253, 253],
         [ 20,  20,  20, ..., 253, 253, 253],
         [ 20,  20,  20, ..., 253, 253, 253]],
 
        [[ 69,  69,  69, ...,   0,   0,   0],
         [ 69,  69,  69, ...,   0,   0,   0],
         [ 69,  69,  69, ..., 211, 211,   0],
         ...,
         [ 69,  69,  69, ..., 211, 211, 211],
         [ 69,  69,  69, ..., 211, 211, 211],
         [ 69,  69,  69, ..., 211, 211, 211]],
 
        [[249, 249, 249, ...,   0,   0,   0],
         [249, 249, 249, ...,   0,   0,   0],
         [249, 249, 249, ...,  39,  39,   0],
         ...,
         [249, 249, 249, ...,  39,  39,  39],
         [249, 249, 249, ...,  39,  39,  39],
         [249, 249, 249, ...,  39,  39,  39]],
 
        [[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ...,

In [13]:
geom

(array([[[ 20,  20,  20, ..., 253, 253, 253],
         [ 20,  20,  20, ..., 253, 253, 253],
         [ 20,  20,  20, ..., 253, 253, 253],
         ...,
         [ 20,  20,  20, ..., 249, 249, 249],
         [ 20,  20,  20, ..., 249, 249, 249],
         [ 20,  20,  20, ..., 249, 249, 249]],
 
        [[ 69,  69,  69, ..., 211, 211, 211],
         [ 69,  69,  69, ..., 211, 211, 211],
         [ 69,  69,  69, ..., 211, 211, 211],
         ...,
         [ 69,  69,  69, ..., 150, 150, 150],
         [ 69,  69,  69, ..., 150, 150, 150],
         [ 69,  69,  69, ..., 150, 150, 150]],
 
        [[249, 249, 249, ...,  39,  39,  39],
         [249, 249, 249, ...,  39,  39,  39],
         [249, 249, 249, ...,  39,  39,  39],
         ...,
         [249, 249, 249, ...,  39,  39,  39],
         [249, 249, 249, ...,  39,  39,  39],
         [249, 249, 249, ...,  39,  39,  39]],
 
        [[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ...,

In [25]:
lc.bounds

BoundingBox(left=4155094.8132666866, bottom=3433673.5335791074, right=4557094.813266687, top=3851093.5335791074)

In [11]:
#lc_a.shape

In [12]:
rgbs = []
for idx in tqdm(range(len(rgb_files))):
    rgb = rio.open(rgb_path + rgb_files[idx])
    rgb_r = rgb.read()
    rgb.close()
    
    rgb_a = align(rgb, lc,rgb_r, shape)
    
    rgb_a = np.moveaxis(rgb_a, 0, -1)
    
    rgbs.append(rgb_a)


 54%|██████████████████████▌                   | 7/13 [02:26<02:05, 20.91s/it]


CPLE_OutOfMemoryError: d:\bld\gdal-split_1623433063894\work\frmts\mem\memdataset.cpp, 1513: cannot allocate 1x1678028400 bytes

In [ ]:
len(rgbs)